In [1]:
import pandas as pd
from datetime import datetime

In [2]:
# Data is loaded from (https://www.data.qld.gov.au/dataset/crash-data-from-queensland-roads)
df = pd.read_csv('e88943c0-5968-4972-a15f-38e120d72ec0.csv')

/var/folders/p3/r3tlf0sj4rsfpv7j1hvsz4vr0000gn/T/ipykernel_58880/1828487975.py:1: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('e88943c0-5968-4972-a15f-38e120d72ec0.csv')


In [3]:
df = df[df['Crash_Year'].between(2019, 2022)]

# Function to convert Crash_Year, Crash_Month, Crash_Day_Of_Week, Crash_Hour to year, month, day, and time
def convert_to_datetime(row):
    year = row['Crash_Year']
    month = row['Crash_Month']
    day = row['Crash_Day_Of_Week']
    hour = row['Crash_Hour']
    
    return year, month, day, f"{hour:02}:00"

# Apply function to each row
df['year'], df['month'], df['day'], df['time'] = zip(*df.apply(convert_to_datetime, axis=1))

# Drop the original columns if needed
df.drop(columns=['Crash_Year', 'Crash_Month', 'Crash_Day_Of_Week', 'Crash_Hour'], inplace=True)

# Display the modified DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51518 entries, 80694 to 386332
Data columns (total 53 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   _id                              51518 non-null  int64  
 1   Crash_Ref_Number                 51518 non-null  int64  
 2   Crash_Severity                   51518 non-null  object 
 3   Crash_Nature                     51518 non-null  object 
 4   Crash_Type                       51518 non-null  object 
 5   Crash_Longitude                  51518 non-null  float64
 6   Crash_Latitude                   51518 non-null  float64
 7   Crash_Street                     51518 non-null  object 
 8   Crash_Street_Intersecting        22523 non-null  object 
 9   State_Road_Name                  23936 non-null  object 
 10  Loc_Suburb                       51518 non-null  object 
 11  Loc_Local_Government_Area        51518 non-null  object 
 12  Loc_Post_Code     

In [4]:
df.rename(columns={'Crash_Ref_Number': 'report_id'}, inplace=True)

# Add a new column 'state' with all values as 'QLD'
df['state'] = 'QLD'


In [5]:
# Mapping dictionary for transformation
remoteness_mapping = {
    'Major Cities': 'City',
    'Inner Regional': 'Metropolitan',
    'Outer Regional': 'Metropolitan',
    'Remote': 'Country',
    'Very Remote': 'Country',
    'unknown': 'Unknown'
}

# Apply mapping to transform 'Loc_ABS_Remoteness' column
df['Loc_ABS_Remoteness'] = df['Loc_ABS_Remoteness'].map(remoteness_mapping)

# Drop rows where 'Loc_ABS_Remoteness' is 'Unknown'
df = df[df['Loc_ABS_Remoteness'] != 'Unknown']

# Generate 'stats_area' column based on transformed values
df['stats_area'] = df['Loc_ABS_Remoteness']


In [6]:
df.rename(columns={'Loc_Suburb': 'lga'}, inplace=True)


In [7]:
# Rename columns 'Crash_Longitude' and 'Crash_Latitude' to 'longitude' and 'latitude'
df.rename(columns={'Crash_Longitude': 'longitude', 'Crash_Latitude': 'latitude'}, inplace=True)

In [8]:
# Determine loc_type based on Crash_Street_Intersecting
df['loc_type'] = df['Crash_Street_Intersecting'].apply(lambda x: 'Intersection' if pd.notna(x) else 'Midblock')

# Combine Crash_Street and Crash_Street_Intersecting into location column
df['location'] = df.apply(lambda row: f"{row['Crash_Street']} & {row['Crash_Street_Intersecting']}" if row['loc_type'] == 'Intersection' else row['Crash_Street'], axis=1)

# Drop the original columns if needed
df.drop(columns=['Crash_Street', 'Crash_Street_Intersecting'], inplace=True)


In [9]:
lighting_mapping = {
    'Daylight': 'Day',
    'Dawn/Dusk': 'Day',
    'Darkness - Not lighted': 'Night',
    'Darkness - Lighted': 'Night',
    'Unknown': None  # Mapping 'Unknown' to None to remove these records
}

# Apply mapping to create light_cond column
df['light_cond'] = df['Crash_Lighting_Condition'].map(lighting_mapping)

# Drop rows where light_cond is None (i.e., remove 'Unknown' records)
df = df.dropna(subset=['light_cond'])

# Drop the original Crash_Lighting_Condition column if needed
df.drop(columns=['Crash_Lighting_Condition'], inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51518 entries, 80694 to 386332
Data columns (total 55 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   _id                              51518 non-null  int64  
 1   report_id                        51518 non-null  int64  
 2   Crash_Severity                   51518 non-null  object 
 3   Crash_Nature                     51518 non-null  object 
 4   Crash_Type                       51518 non-null  object 
 5   longitude                        51518 non-null  float64
 6   latitude                         51518 non-null  float64
 7   State_Road_Name                  23936 non-null  object 
 8   lga                              51518 non-null  object 
 9   Loc_Local_Government_Area        51518 non-null  object 
 10  Loc_Post_Code                    51518 non-null  object 
 11  Loc_Police_Division              51518 non-null  object 
 12  Loc_Police_Distric

In [11]:
# Mapping dictionary for atmospheric conditions
atmospheric_mapping = {
    'Clear': 'Not Raining',
    'Fog': 'Not Raining',
    'Smoke/Dust': 'Not Raining',
    'Raining': 'Raining',
    'Unknown': None  # Mapping 'Unknown' to None to remove these records
}

# Apply mapping to create new column
df['weather_cond'] = df['Crash_Atmospheric_Condition'].map(atmospheric_mapping)

# Drop rows where atmospheric_cond is None (i.e., remove 'Unknown' records)
df = df.dropna(subset=['weather_cond'])

# Drop the original Crash_Atmospheric_Condition column if needed
df.drop(columns=['Crash_Atmospheric_Condition'], inplace=True)


In [12]:
# Mapping dictionary for speed limits
speed_limit_mapping = {
    '0 - 50 km/h': 50,
    '60 km/h': 60,
    '70 km/h': 70,
    '80 - 90 km/h': 90,
    '100 - 110 km/h': 110,
    '': None  # Mapping blank values to None to remove these records
}

# Apply mapping to create speed_limit column
df['speed_limit'] = df['Crash_Speed_Limit'].map(speed_limit_mapping)

# Drop rows where speed_limit is None (i.e., remove blank entries)
df = df.dropna(subset=['speed_limit'])

# Drop the original Crash_Speed_Limit column if needed
df.drop(columns=['Crash_Speed_Limit'], inplace=True)

In [13]:
# Mapping dictionary for severity levels
severity_mapping = {
    'Fatal': '4: Fatal',
    'Hospitalisation': '3: SI',
    'Medical treatment': '2: MI',
    'Minor injury': '1: PDO',
    'Unknown': None  # Mapping 'Unknown' to None to handle undefined cases
}

df['csef_severity'] = df['Crash_Severity'].map(severity_mapping)

# Drop the original Crash_Severity column if needed
df.drop(columns=['Crash_Severity'], inplace=True)


In [14]:
# List of standardized columns to keep
columns_to_keep = [
    'report_id',
    'year',
    'month',
    'day',
    'time',
    'state',
    'stats_area',
    'lga',
    'latitude',
    'longitude',
    'loc_type',
    'location',
    'light_cond',
    'weather_cond',
    'speed_limit',
    'csef_severity'
]

# Select only the specified columns
df = df[columns_to_keep]

In [15]:
df.to_csv('Final_QLD.csv', index=False)